# Try out [the new subplot approach](https://github.com/holoviz/holoviews/pull/5840)

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np

import holoviews as hv; hv.extension('bokeh')
from holoviews.plotting.links import RangeToolLink
from holoviews.operation.datashader import rasterize
from holoviews import Dataset

from neurodatagen.eeg import generate_eeg_powerlaw
from hvneuro import download_file

from bokeh.models import HoverTool
from scipy.stats import zscore
import panel as pn; pn.extension(template='material')

In [ ]:
n_channels = 10
n_seconds = 5
fs = 512

data, time, channels = generate_eeg_powerlaw(n_channels, n_seconds, fs)

print(f'shape: {data.shape} (n_channels, samples) ')
data

### Visualize synthetic data

In [ ]:
channel_curves = []
ch_subcoord = 1./n_channels
for i, channel_data in enumerate(data):
    
    channel_curves.append(hv.Curve(channel_data).opts(
            subcoordinate_y=(i*ch_subcoord, (i+1)*ch_subcoord), color="black", line_width=1, 
            tools=['hover', 'xwheel_zoom'], shared_axes=False))

eeg_viewer = hv.Overlay(channel_curves, kdims="Channel").opts(width=800, height=500)
eeg_viewer

## my shared version

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv; hv.extension('bokeh')
from holoviews import opts
from holoviews import Dataset
from holoviews.plotting.links import RangeToolLink
from bokeh.models import HoverTool
import colorcet as cc
import panel as pn; pn.extension()
from scipy.stats import zscore

n_channels = 10
n_seconds = 5
fs = 512
max_ch_disp = 5  # max channels to initially display
max_t_disp = 3 # max time in seconds to initially display

total_samples = n_seconds * fs
time = np.linspace(0, n_seconds, total_samples)
data = np.random.randn(n_channels, total_samples).cumsum(axis=1)
channels = ['EEG {}'.format(i) for i in range(n_channels)]

channel_curves = []
ch_subcoord = 1./n_channels

hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")])

for i, channel_data in enumerate(data):
    ds = Dataset((time, channel_data, channels[i]), ["Time", "Amplitude", "channel"])
    curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=channels[i]).opts(
            subcoordinate_y=(i*ch_subcoord, (i+1)*ch_subcoord), color="black", line_width=1, 
            tools=[hover, 'xwheel_zoom'], shared_axes=False)
    channel_curves.append(curve)

eeg_viewer = hv.Overlay(channel_curves, kdims="Channel").opts(
    padding=0, xlabel="Time (s)", ylabel="Channel",
    show_legend=False, aspect=1.5, responsive=True, # yticks=yticks,
    shared_axes=False, backend_opts={
        "x_range.bounds": (time.min(), time.max()),
        "y_range.bounds": (data.min(), data.max())})

# # Create events
# events = [
#     {'start': 1, 'end': 1.5, 'label': 'event1'},
#     {'start': 2, 'end': 2.5, 'label': 'event1'},
#     {'start': 3, 'end': 3.5, 'label': 'event1'},
#     {'start': 1.5, 'end': 2, 'label': 'event2'},
#     {'start': 2.5, 'end': 3, 'label': 'event2'},
# ]

# labels = list(set([event['label'] for event in events]))
# colors = cc.glasbey_bw_minc_20_minl_30[:len(labels)]
# label_colors = dict(zip(labels, colors))

# for event in events:
#     vspan = hv.VSpan(event['start'], event['end']).opts(fill_alpha=0.5, fill_color=label_colors[event['label']])
#     eeg_viewer *= vspan

yticks = [( (i*ch_subcoord + (i+1)*ch_subcoord) / 2, ich) for i, ich in enumerate(channels)]
y_positions, _ = zip(*yticks)

z_data = zscore(data, axis=1)

minimap = hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=100, responsive=True, default_tools=[''], shared_axes=False, clim=(-z_data.std()*2.5, z_data.std()*2.5))


if len(channels) < max_ch_disp:
    max_ch_disp = len(channels)
max_y_disp = (max_ch_disp+2)*ch_subcoord

time_s = len(time)/fs
if time_s < max_t_disp:
    max_t_disp = time_s
    
RangeToolLink(minimap, list(eeg_viewer.values())[0], axes=["x", "y"],
              boundsx=(None, max_t_disp),
              boundsy=(None, max_y_disp))

# eeg_app = pn.Column(pn.Row(eeg_viewer, min_height=500), minimap).servable(target='main')
eeg_app = pn.Column((eeg_viewer + minimap).cols(1), min_height=650).servable(target='main')
eeg_app

## philipps version after pr work

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv; hv.extension('bokeh')
from holoviews import opts
from holoviews import Dataset
from holoviews.plotting.links import RangeToolLink
from bokeh.models import HoverTool
import panel as pn; pn.extension()
from scipy.stats import zscore

n_channels = 10
n_seconds = 5
fs = 512
max_ch_disp = 5  # max channels to initially display
max_t_disp = 3 # max time in seconds to initially display

total_samples = n_seconds * fs
time = np.linspace(0, n_seconds, total_samples)
data = np.random.randn(n_channels, total_samples).cumsum(axis=1)
channels = ['EEG {}'.format(i) for i in range(n_channels)]

channel_curves = []
ch_subcoord = 1./n_channels

hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")])

for i, channel_data in enumerate(data):
    ds = Dataset((time, channel_data, channels[i]), ["Time", "Amplitude", "channel"])
    channel_curves.append(hv.Curve(ds, "Time", ["Amplitude", "channel"], label=channels[i]).opts(
            subcoordinate_y=(i*ch_subcoord, (i+1)*ch_subcoord), color="black", line_width=1, 
            tools=[hover, 'xwheel_zoom'], shared_axes=False))

annotation = hv.VSpan(0.3, 0.5)
eeg_viewer = (hv.Overlay(channel_curves, kdims="Channel") * annotation).opts(
    padding=0, xlabel="Time (s)", ylabel="Channel",
    show_legend=False, aspect=1.5, min_height=500, responsive=True,
    shared_axes=False, backend_opts={
        "x_range.bounds": (time.min(), time.max()),
        "y_range.bounds": (0, 1)}) 

yticks = [( (i*ch_subcoord + (i+1)*ch_subcoord) / 2, ich) for i, ich in enumerate(channels)]
y_positions, _ = zip(*yticks)

z_data = zscore(data, axis=1)

minimap = hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=100, responsive=True, default_tools=[''], shared_axes=False, clim=(-z_data.std()*2.5, z_data.std()*2.5))

if len(channels) < max_ch_disp:
    max_ch_disp = len(channels)
max_y_disp = (max_ch_disp+2)*ch_subcoord

time_s = len(time)/fs
if time_s < max_t_disp:
    max_t_disp = time_s
    
RangeToolLink(minimap, eeg_viewer, axes=["x", "y"],
              boundsx=(None, max_t_disp),
              boundsy=(None, max_y_disp))

eeg_app = pn.Column((eeg_viewer + minimap * annotation).cols(1), min_height=650).servable(target='main')
eeg_app

### Bokeh

In [ ]:
from bokeh.models import ColumnDataSource, Range1d, RangeTool
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.palettes import Viridis10
from bokeh.transform import linear_cmap
from bokeh.io import output_notebook, show; output_notebook()
import numpy as np
from scipy.stats import zscore

n_channels = 10
n_seconds = 5
fs = 512
total_samples = n_seconds * fs
time = np.linspace(0, n_seconds, total_samples)
data = np.random.randn(n_channels, total_samples).cumsum(axis=1)
channels = ['EEG {}'.format(i) for i in range(n_channels)]

p = figure(height=500, width=900, tools="xwheel_zoom,xpan,xbox_zoom,reset", active_scroll="xwheel_zoom")

for i, channel_data in enumerate(data):
    y_range = Range1d(start=i, end=i+1)
    subplot = p.subplot(x_source=p.x_range, x_target=p.x_range, y_source=p.y_range, y_target=y_range)
    source = ColumnDataSource(data=dict(x=time, y=channel_data, channel=[channels[i]]*len(time)))
    subplot.line('x', 'y', source=source, line_color="black", line_width=1, alpha=0.7)

# p.yaxis.ticker = list(range(n_channels))
# p.yaxis.major_label_overrides = {i: channel for i, channel in enumerate(channels)}
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Channel'
p.grid.visible = False

show(p)

### Original

In [ ]:
max_ch_disp = 10  # max channels to initially display
max_t_disp = 4 # max time in seconds to initially display

spacing = 2.5  # Spacing between channels
offset = np.std(data) * spacing

# Create a hv.Curve element per chan.
# Note: alternative is to call hv.Path once on offset-adjusted data, but 
# then we couldn't independently apply formating to the channels (which 
# we aren't doing yet, but we likely will in the future)
channel_curves = []
max_data = data.max()
hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "@original_amplitude µV")])
for i, channel_data in enumerate(data):
    offset_data = channel_data + (i * offset)
    max_data = max(offset_data.max(), max_data) # update max
    ds = Dataset((time, offset_data, channel_data, channels[i]), ["Time", "Amplitude", "original_amplitude", "channel"])
    channel_curves.append(
        hv.Curve(ds, "Time", ["Amplitude", "original_amplitude", "channel"]).opts(
            color="black", line_width=1,
            tools=[hover, 'xwheel_zoom'], shared_axes=False))

# Create mapping from yaxis location to ytick for each channel
# so we can have categorical-style labeling on a continuous axis.
# Note: this would/should change when we implement independent 
# coordinates.
yticks = [(i * offset, ich) for i, ich in enumerate(channels)]


# Create an overlay of curves
# TODO.. setting x/y_range bounds does not yet restrict the RangeTool from going beyond these limits
# TODO.. the zoom out will stop when it hits any single bound, and not continue zooming out in other directions/dims
eeg_viewer = hv.Overlay(channel_curves, kdims="Channel").opts(
    padding=0, xlabel="Time (s)", ylabel="Channel", #default_tools=['hover', 'pan', 'box_zoom', 'save', 'reset'],
    yticks=yticks, show_legend=False, aspect=1.5, responsive=True,
    shared_axes=False, backend_opts={
        "x_range.bounds": (time.min(), time.max()),
        "y_range.bounds": (data.min(), max_data)})


# # Create events
# events = [
#     {'start': 1, 'end': 1.5, 'label': 'event1'},
#     {'start': 2, 'end': 2.5, 'label': 'event1'},
#     {'start': 3, 'end': 3.5, 'label': 'event1'},
#     {'start': 1.5, 'end': 2, 'label': 'event2'},
#     {'start': 2.5, 'end': 3, 'label': 'event2'},
# ]

# labels = list(set([event['label'] for event in events]))
# colors = cc.glasbey_category10[:len(labels)]
# label_colors = dict(zip(labels, colors))

# for event in events:
#     vspan = hv.VSpan(event['start'], event['end']).opts(responsive=True, fill_alpha=0.5, fill_color=tuple(label_colors[event['label']]))
#     eeg_viewer *= vspan

# Get the y positions of the yticks to use as yaxis of minimap image
y_positions, _ = zip(*yticks)

# Compute z-scores across time for each channel
z_data = zscore(data, axis=1)

# Generate the zscored image for the minimap using the y tiack positions from the eeg_viewer
minimap = hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")

# RangeTool doesn't work with rasterized object? TODO: file issue
# minimap = rasterize(minimap)

# maybe I should datashade/2d-bin the data before creating the hv.Image
# I could use lttb (1d so per channel) or ResampleOperation2D (but I think that applies to the entire nb)
# or some operation from datashader to return the 2d hist


# Style the minimap 
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=100, responsive=True, default_tools=[''], shared_axes=False, clim=(-z_data.std()*2.5, z_data.std()*2.5))
    
# Create RangeToolLink between the minimap and the main EEG viewer 
# (quirk: apply to just one eeg trace and it will apply to all. see HoloViews #4472)
max_y_disp = np.max(data[max_ch_disp-1,:] + ((max_ch_disp-1) * offset))
RangeToolLink(minimap, list(eeg_viewer.values())[0], axes=["x", "y"],
              boundsx=(None, max_t_disp),
              boundsy=(None, max_y_disp))

# Display vertically
# layout = (eeg_viewer + minimap).cols(1).opts(shared_axes=False, merge_tools=False)
# eeg_app = pn.Row(layout).servable() # too much spacing between plots in served app
# eeg_app = pn.Column(pn.Row(eeg_viewer, min_height=500, sizing_mode='stretch_both'), minimap, sizing_mode='stretch_both')#.servable()#target='main') # BUG Panel #5315: rangetool is variably active in the bokeh toolbar on eeg viewer plot.. not respecting shared_axes=False
# eeg_app
eeg_app = pn.Column((eeg_viewer + minimap).cols(1), min_height=650).servable(target='main')
eeg_app

## Real data pipeline

In [ ]:
import mne

### Intake data

In [ ]:
# This dataset is 2.6 MB on disk
url = "https://physionet.org/files/eegmmidb/1.0.0/S001/S001R04.edf?download"
local_data_path = "~/data/mne_data"


# Will not download if already present at local_data_path
local_file_path = download_file(url, local_data_path)

In [ ]:
raw = mne.io.read_raw_edf(local_file_path, preload=True)
raw.info

In [ ]:
# preview the channel names, types, signal ranges, and uncompressed size
raw.describe()

### Clean channel names, set sensor positions, and reference data

In [ ]:
# clean up the channel names
raw.rename_channels(lambda s: s.strip("."));

In [ ]:
# preview available montages that are shipped with MNE
# mne.channels.get_builtin_montages(descriptions=True)

In [ ]:
# Let's use the standard 10-20
montage = mne.channels.make_standard_montage("standard_1020")

In [ ]:
# plot the assigned positions of our data channels
raw.set_montage(montage, match_case=False)
sphere=(0, 0.015, 0, 0.099) #manually adjust the y origin coord and radius a bit
raw.plot_sensors(show_names=True, sphere=sphere);

In [ ]:
# re-reference EEG data to the average over all recording channels
raw.set_eeg_reference("average");

### Gather the data for plotting into simple arrays

In [ ]:
time = raw.times
channels = raw.ch_names

# get the EEG data (for this data set, all channels are EEG anyways)
eeg_indices = mne.pick_types(raw.info, eeg=True)
data = raw.get_data(picks=eeg_indices, units={"eeg":"uV"})

In [ ]:
# data = data[:5,600:1600]

### Visualize real data

#### Philipp's new version with real data

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv; hv.extension('bokeh')
from holoviews import opts
from holoviews import Dataset
from holoviews.plotting.links import RangeToolLink
from bokeh.models import HoverTool
import panel as pn; pn.extension()
from scipy.stats import zscore

n_channels = len(channels)
# n_seconds = 5
fs = raw.info['sfreq']
max_ch_disp = 5  # max channels to initially display
max_t_disp = 3 # max time in seconds to initially display

# total_samples = n_seconds * fs
# time = np.linspace(0, n_seconds, total_samples)
# data = np.random.randn(n_channels, total_samples).cumsum(axis=1)
# channels = ['EEG {}'.format(i) for i in range(n_channels)]

channel_curves = []
ch_subcoord = 1./n_channels

hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")])

for i, channel_data in enumerate(data):
    ds = Dataset((time, channel_data, channels[i]), ["Time", "Amplitude", "channel"])
    channel_curves.append(hv.Curve(ds, "Time", ["Amplitude", "channel"], label=channels[i]).opts(
            subcoordinate_y=((i)*(1./n_channels), (i+10)*(1./n_channels)), color="black", line_width=1, 
            tools=[hover, 'xwheel_zoom'], shared_axes=False))

annotation = hv.VSpan(0.3, 0.5)
eeg_viewer = (hv.Overlay(channel_curves, kdims="Channel") * annotation).opts(
    padding=0, xlabel="Time (s)", ylabel="Channel",
    show_legend=False, aspect=1.5, min_height=500, responsive=True,
    shared_axes=False, backend_opts={
        "x_range.bounds": (time.min(), time.max()),
        "y_range.bounds": (0, 1)}) 

yticks = [( (i*ch_subcoord + (i+1)*ch_subcoord) / 2, ich) for i, ich in enumerate(channels)]
y_positions, _ = zip(*yticks)

z_data = zscore(data, axis=1)

minimap = hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=100, responsive=True, default_tools=[''], shared_axes=False, clim=(-z_data.std()*2.5, z_data.std()*2.5))

minimap = rasterize(minimap)

if len(channels) < max_ch_disp:
    max_ch_disp = len(channels)
max_y_disp = (max_ch_disp+2)*ch_subcoord

time_s = len(time)/fs
if time_s < max_t_disp:
    max_t_disp = time_s
    
RangeToolLink(minimap, eeg_viewer, axes=["x", "y"],
              boundsx=(None, max_t_disp),
              boundsy=(None, max_y_disp))

eeg_app = pn.Column((eeg_viewer + minimap * annotation).cols(1), min_height=650).servable(target='main')
eeg_app

In [ ]:
max_ch_disp = 10  # max channels to initially display
max_t_disp = 5 # max time in seconds to initially display

n_channels = data.shape[0]
channel_curves = []
ch_subcoord = 1./n_channels

hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")])

for i, channel_data in enumerate(data):
    ds = Dataset((time, channel_data, channels[i]), ["Time", "Amplitude", "channel"])
    channel_curves.append(hv.Curve(ds, "Time", ["Amplitude", "channel"], label=channels[i]).opts(
            subcoordinate_y=((i-1)*ch_subcoord, (i+2)*ch_subcoord), color="black", line_width=1, 
            tools=[hover, 'xwheel_zoom'], shared_axes=False))

eeg_viewer = hv.Overlay(channel_curves, kdims="Channel").opts(responsive=True, ylabel="Channel")

yticks = [(((i+2)*ch_subcoord)+((i-1)*ch_subcoord/2), ich) for i, ich in enumerate(channels)]
y_positions, _ = zip(*yticks)

z_data = zscore(data, axis=1)

minimap = hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=100, responsive=True, default_tools=[''], shared_axes=False, clim=(-z_data.std()*2.5, z_data.std()*2.5))


if len(channels) < max_ch_disp:
    max_ch_disp = len(channels)
max_y_disp = (max_ch_disp+2)*ch_subcoord

time_s = len(time)/raw.info['sfreq']
if time_s < max_t_disp:
    max_t_disp = time_s
    
RangeToolLink(minimap, list(eeg_viewer.values())[0], axes=["x", "y"],
              boundsx=(None, max_t_disp),
              boundsy=(None, max_y_disp))

eeg_app = pn.Column(pn.Row(eeg_viewer, min_height=500), minimap).servable(target='main')
eeg_app

In [ ]:
max_ch_disp = 10  # max channels to initially display
max_t_disp = 5 # max time in seconds to initially display

spacing = 2.5  # Spacing between channels
offset = np.std(data) * spacing

# Create a hv.Curve element per channel
channel_curves = []
max_data = data.max()
hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "@original_amplitude µV")])
for i, channel_data in enumerate(data):
    offset_data = channel_data + (i * offset)
    max_data = max(offset_data.max(), max_data)  # update max
    ds = Dataset((time, offset_data, channel_data, channels[i]), ["Time", "Amplitude", "original_amplitude", "channel"])
    channel_curves.append(
        hv.Curve(ds, "Time", ["Amplitude", "original_amplitude", "channel"]).opts(
            color="black", line_width=1,
            tools=[hover, 'xwheel_zoom'], shared_axes=False))

# Create mapping from yaxis location to ytick for each channel
yticks = [(i * offset, ich) for i, ich in enumerate(channels)]

# Create an overlay of curves
eeg_viewer = hv.Overlay(channel_curves, kdims="Channel").opts(
    padding=0, xlabel="Time (s)", ylabel="Channel", yticks=yticks, show_legend=False, aspect=1.5, responsive=True,
    shared_axes=False, backend_opts={
        "x_range.bounds": (time.min(), time.max()),
        "y_range.bounds": (data.min(), max_data)})

# Get the y positions of the yticks to use as yaxis of minimap image
y_positions, _ = zip(*yticks)

# Compute z-scores across time for each channel
z_data = zscore(data, axis=1)

# Generate the zscored image for the minimap using the y tick positions from the eeg_viewer
minimap = hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=100, responsive=True, default_tools=[''], shared_axes=False, clim=(-z_data.std()*2.5, z_data.std()*2.5))

# Create RangeToolLink between the minimap and the main EEG viewer 
if len(channels) < max_ch_disp:
    max_ch_disp = len(channels)
max_y_disp = np.max(data[max_ch_disp-1,:] + ((max_ch_disp-1) * offset))

time_s = len(time)/raw.info['sfreq']
if time_s < max_t_disp:
    max_t_disp = time_s
    
RangeToolLink(minimap, list(eeg_viewer.values())[0], axes=["x", "y"],
              boundsx=(None, max_t_disp),
              boundsy=(None, max_y_disp))

eeg_app = pn.Column(pn.Row(eeg_viewer, min_height=500), minimap).servable(target='main')
eeg_app